In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os 
os.chdir("/content/drive/My Drive/fast.ai v3/section 2/cust_dl_lib")

!git config --global user.email "rajp152000@gmail.com"
!git config --global user.name "rajp152k"
!pip install fire
import fire
datasets_path = "/content/drive/My Drive/fast.ai v3/datasets"

     |████████████████████████████████| 81kB 2.4MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=c564812f594b2663fc9baa7621ee0c737a3a8dd0b7fa9042fdc70e7271e9b924
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
Successfully built fire


alternative to kaiming init for better maintenance of means and variances than the control model(randomly initialised)

In [0]:
#export 
from exps.nb_07 import *

# LSUV
### layerwise sequential unit variance 

getting data

In [0]:
x_train,y_train,x_valid,y_valid = get_data()

x_train,x_valid = normalize_to(x_train,x_valid)
train_ds,valid_ds = Dataset(x_train,y_train),Dataset(x_valid,y_valid)

nh,bs = 50,512
c = y_train.max().item() + 1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds,valid_ds,bs),c)

In [0]:
mnist_view = view_tfm(1,28,28)
cbfs = [Recorder,
        partial(AvgStatsCallback,accuracy),
        CudaCallback,
        partial(BatchTransformXCallback,mnist_view)]

In [0]:
nfs = [8,16,32,64,64]

In [0]:
class ConvLayer(nn.Module):
    def __init__(self,ni,nf,ks=3,stride=2,sub=0.,**kwargs):
        super().__init__()
        self.conv = nn.Conv2d(ni,nf,ks,stride=stride,bias=True,padding =ks//2)
        self.relu = GeneralRelu(sub=sub,**kwargs)

    def forward(self,x):
        return self.relu(self.conv(x))

    @property
    def bias(self) : return -self.relu.sub
    @bias.setter
    def bias(self,v): self.relu.sub = -v
    @property
    def weight(self): return self.conv.weight

In [0]:
learn,run = get_learn_run(nfs,data,0.6,ConvLayer,cbs=cbfs)

progression:
 - initialize normally
 - pass a batch
 - observe stats of activations 
 - rescale weights according to activations' variances
 - subtract activations' means from weights' old bias.
 - repeat till stabilization

control run

In [13]:
run.fit(2,learn)

train: [1.46421515625, tensor(0.4900, device='cuda:0')]
valid: [0.205481982421875, tensor(0.9391, device='cuda:0')]
train: [0.2610046875, tensor(0.9219, device='cuda:0')]
valid: [0.3124569091796875, tensor(0.9130, device='cuda:0')]


now using LSUV on a test model

In [0]:
learn,run = get_learn_run(nfs,data,0.6,ConvLayer,cbfs)

defining a func to extract one batch after being processed by the callbacks

In [0]:
#export 
def get_batch(dl,run):
    run.xb,run.yb = next(iter(dl))
    for cb in run.cbs:cb.set_runner(run)
    run('begin_batch')
    return run.xb,run.yb

In [0]:
xb,yb = get_batch(data.train_dl,run)

only want to perform LSUV changes on linear and convolutional layers

In [0]:
#export 
def find_modules(m,cond):
    if cond(m): return [m]
    return sum([find_modules(o,cond) for o in m.children()],[])

read about sum's behaviour on lists:
if just a normal concatenation:
we get residual empty modules in the end of the  assignee to the function

recursively tests for the module in m(nn.Sequential object) satistfying the condition "cond"

cond for convlayers is simple: 
use isinstance of the convlayer class created before

writing cond for a generic set of layers which we might have incorporated except the written ConvLayer class

In [0]:
#export 
def is_lin_layer(l):
    lin_layers = (nn.Conv1d,nn.Conv2d,nn.Conv3d,nn.Linear,nn.ReLU)
    return is_instance(l,lin_layers)

getting the corresponding modules

In [0]:
mods = find_modules(learn.model,lambda o: isinstance(o,ConvLayer))

In [57]:
mods

[ConvLayer(
   (conv): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 )]

hooks for stats

In [0]:
def append_stat(hook,mod,inp,outp):
    d = outp.data
    hook.mean,hook.std = d.mean().item(),d.std().item()

In [0]:
mdl = learn.model.cuda()

In [63]:
with Hooks(mods,append_stat) as hooks:
    mdl(xb)
    for hook in hooks: print(hook.mean,hook.std)

0.5601759552955627 0.9255796670913696
0.6497805118560791 0.9860087037086487
0.554811418056488 0.9236533641815186
0.45467084646224976 0.7147250175476074
0.27226799726486206 0.4576934278011322


expected :<br>
stds are decreasing exponentially and menas are no good either

creating the lsuv module:
 - first adjust the bias using the means 
 - then adjust the stds to unity

In [0]:
#export 
def lsuv_module(m,xb):
    h = Hook(m,append_stat)

    while mdl(xb) is not None and abs(h.mean) > 1e-3: m.bias-=h.mean
    while mdl(xb) is not None and abs(h.std-1) >1e-3: m.weight.data/=h.std

    h.remove()
    return h.mean,h.std

In [69]:
for m in mods: print(lsuv_module(m,xb))

(0.04504041746258736, 1.0000001192092896)
(0.02088899537920952, 0.9999999403953552)
(0.16514232754707336, 1.0000001192092896)
(0.17346076667308807, 0.9999998807907104)
(0.2991844117641449, 0.9999999403953552)


better

eventhough the mean quite close to zero, stds are 1: wont cause explosions/collapses of activations

checking the model's performance

In [71]:
%time run.fit(2,learn)

train: [0.4860132421875, tensor(0.8425, device='cuda:0')]
valid: [0.127663232421875, tensor(0.9610, device='cuda:0')]
train: [0.108801435546875, tensor(0.9659, device='cuda:0')]
valid: [0.09362228393554688, tensor(0.9720, device='cuda:0')]
CPU times: user 1.76 s, sys: 335 ms, total: 2.1 s
Wall time: 2.11 s


LSUV is good for deeper layers where it's difficult to maintain unit variance throughout : allowing for healthier activations and faster training as observed above

# Exporting

In [0]:
!python3 notebook2script.py 07a_LSUV.ipynb

Converted 07a_LSUV.ipynb to exps/nb_07a.py
